In [15]:
import pandas as pd
from bokeh.plotting import figure, show

usa_census = pd.read_csv(
    './data/USA/DECENNIALDDHCA2020.T01001_2024-12-15T154541/DECENNIALDDHCA2020.T01001-Data.csv'
)

usa_census.drop(index=usa_census.index[0], axis=0, inplace=True)

usa_census.drop(usa_census.columns[5], axis=1, inplace=True)

# usa_census.head(100)


# p = figure(title="Hello Plot!", x_axis_label='blah', y_axis_label='foo')


,Column Name,Label
0,GEO_ID,Geography
1,NAME,Geographic Area Name
2,T01001_001N,!!Total
3,POPGROUP,Race/Ethnic Group
4,POPGROUP_LABEL,Population Groups
